In [1]:
module O

export rand_LCG, seed_LCG!

# Linear congruential generators (LCGs)
# Parameters are provided by Park and Miller
# See https://c-faq.com/lib/rand.html

mutable struct LinearCongruentialGenerator <: Function
    seed::Int32
end

const a = Int32(48271)
const m = Int32(2147483647)
const q = m ÷ a
const r = m % a

function (f::LinearCongruentialGenerator)()
    seed = f.seed
    hi, lo = divrem(seed, q)
    seed = a * lo - r * hi
    seed = ifelse(seed > zero(seed), seed, seed + m)
    f.seed = seed
    seed / m
end

const rand_LCG = LinearCongruentialGenerator(Int32(20231226))

seed_LCG!(seed) = rand_LCG.seed = Int32(seed)

end

using .O

In [2]:
function mcpi_LCG(num_points = 10^9, seed = 20231226)
    seed_LCG!(seed)
    num_inside = 0
    for i in 1:num_points
        num_inside += rand_LCG()^2 + rand_LCG()^2 < 1
    end
    4num_inside / num_points
end

@time mcpi_LCG()
@time mcpi_LCG()
@time mcpi_LCG()

  8.853515 seconds
  8.874661 seconds
  8.957782 seconds


3.141546348

In [3]:
using LoopVectorization

isinside(i) = rand_LCG()^2 + rand_LCG()^2 < 1

function mcpi_LCG_turbo(num_points = 10^9, seed = 20231226)
    seed_LCG!(seed)
    num_inside = 0
    @turbo for i in 1:num_points
        num_inside += isinside(i)
    end
    4num_inside / num_points
end

@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()
@time mcpi_LCG_turbo()

  1.107365 seconds
  1.098373 seconds
  1.102564 seconds


3.141605344